# A notebook to test the model and infrastructure

Here we're going to use the copy stencil to test out the NDSL infrastructure and our ability to run pyFV3 code

## First set up parameters and our MPI environment:

In [ ]:
nx = 20
ny = 20
nz = 10
nhalo = 3
backend = "numpy"

import ipyparallel as ipp

layout = (1, 1)
ntiles = 6
# spinup cluster of MPI-workers
num_ranks = ntiles * layout[0] * layout[1]

cluster = ipp.Cluster(engines="mpi", n=num_ranks).start_and_connect_sync()

# broadcast configuration to all workers
ar = cluster[:].push(
    {
        "ntiles": ntiles,
        "nx": nx,
        "ny": ny,
        "nz": nz,
        "nhalo": nhalo,
        "layout": layout,
        "backend": backend,
    }
)

# start executing cells on the workers in parallel from here on
%autopx

In [ ]:
from mpi4py import MPI

mpi_comm = MPI.COMM_WORLD
mpi_rank = mpi_comm.Get_rank()
print(f"Hello from rank {mpi_rank}")

## Next set up the NDSL structures used by the model:

In [ ]:
import gt4py.cartesian.gtscript as gtscript
from gt4py.cartesian.gtscript import PARALLEL, computation, interval

from ndsl.dsl.typing import Float, FloatField, FloatFieldIJ
from ndsl import (
    CompilationConfig,
    CubedSphereCommunicator,
    CubedSpherePartitioner,
    GridIndexing,
    Quantity,
    QuantityFactory,
    StencilConfig,
    StencilFactory,
    SubtileGridSizer,
    TilePartitioner,
    WrappedHaloUpdater,
)
from ndsl.typing import Communicator
from ndsl.constants import X_DIM, Y_DIM, Z_DIM

In [ ]:
partitioner = CubedSpherePartitioner(TilePartitioner(layout))
cs_communicator = CubedSphereCommunicator(mpi_comm, partitioner)

sizer = SubtileGridSizer.from_tile_params(
    nx_tile=nx,
    ny_tile=ny,
    nz=nz,
    n_halo=nhalo,
    extra_dim_lengths={},
    layout=layout,
    tile_partitioner=partitioner.tile,
    tile_rank=cs_communicator.tile.rank,
)

# useful for easily allocating distributed data storages (fields)
quantity_factory = QuantityFactory.from_backend(sizer=sizer, backend=backend)

compilation_config = CompilationConfig(backend=backend, communicator=cs_communicator)

stencil_config = StencilConfig(compare_to_numpy=False, compilation_config=compilation_config)

grid_indexing = GridIndexing.from_sizer_and_communicator(sizer=sizer, comm=cs_communicator)

stencil_factory = StencilFactory(config=stencil_config, grid_indexing=grid_indexing)

## Set up fields and the stencil:

In [ ]:
from pyFV3.stencils.basic_operations import copy_defn

from_field = quantity_factory.ones(dims=(X_DIM, Y_DIM, Z_DIM), units="none", dtype="float")
to_field = quantity_factory.zeros(dims=(X_DIM, Y_DIM, Z_DIM), units="none", dtype="float")

if mpi_rank == 0:
    print(f"Size of from field on rank 0 is {from_field.extent}")

copy_stencil = stencil_factory.from_origin_domain(
    func = copy_defn,
    origin=grid_indexing.origin_compute(),
    domain=grid_indexing.domain_compute(),
)

## And run everything:

In [ ]:
if mpi_rank in [0, 3]:
    copy_stencil(from_field, to_field)
    print (to_field.data[:,:,0])

There should be two fields shown with 0 in the halo and 1 in the domain

In [ ]:
# req = cs_communicator.start_halo_update(to_field, nhalo)
# req.wait()
cs_communicator.halo_update(to_field, nhalo)
if mpi_rank == 1:
    print (to_field.data[:,:,0])

There should be one field shown with 1 on the left and right halo edges and 0 everywhere else